In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import preprocessing 

In [3]:
#raw_data=np.genfromtxt('C:/Users/ACER M5/Downloads/titanic/train.csv',delimiter=',',usecols=[0,1,2,3,4,5,6,7,8,9,10,11])
raw_data=pd.DataFrame(pd.read_csv('train.csv'))
unscaled_inputs=raw_data.iloc[1:,[2,4,5,6,7,9]]
unscaled_targets=raw_data.iloc[1:,1]
unscaled_test_data=pd.DataFrame(pd.read_csv('test.csv')).iloc[1:,[1,3,4,5,6,8]]
#raw_data2=pd.DataFrame(unscaled_inputs)
#raw_data3=pd.DataFrame(unscaled_targets)
#raw_data2


In [3]:
num_one_targets=int(np.sum(unscaled_targets))

num_zero_targets=0

indices_to_remove=[]

for i in range(unscaled_targets.shape[0]):
    if unscaled_targets.iloc[i]== 0:
         num_zero_targets += 1.
    
         if num_zero_targets > num_one_targets:
                   indices_to_remove.append(i)
    
    
unscaled_inputs_equal_priors=unscaled_inputs.drop(indices_to_remove,axis=0)
unscaled_targets_equal_priors=unscaled_targets.drop(indices_to_remove,axis=0)

In [4]:
scaled_inputs=pd.DataFrame(preprocessing.scale(unscaled_inputs_equal_priors))
scaled_test_data=pd.DataFrame(preprocessing.scale(unscaled_test_data))

shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs.iloc[shuffled_indices]
shuffled_targets = unscaled_targets_equal_priors.iloc[shuffled_indices]

#shuffled_indices_for_test_data=[]

#for i in range(400):
     #if shuffled_indices[i] <= unscaled_test_data.shape[0]:
            #shuffled_indices_for_targets.append(i)
            
#shuffled_test_data=unscaled_test_data.iloc[shuffled_indices_for_test_data]





C:\Users\ACER M5\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.
C:\Users\ACER M5\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by the scale function.
  


In [5]:
sample_count=shuffled_inputs.shape[0]

train_sample_count=int(0.8 * sample_count)
validation_sample_count=int(0.2*sample_count)

train_inputs=shuffled_inputs.iloc[:train_sample_count]
train_targets=shuffled_targets.iloc[:train_sample_count]

validation_inputs=shuffled_inputs.iloc[train_sample_count:train_sample_count+validation_sample_count]
validation_targets=shuffled_targets.iloc[train_sample_count:train_sample_count+validation_sample_count]

test_sample_count=scaled_test_data.shape[0]
test_inputs=scaled_test_data
#test_targets=shuffled_targets.iloc[:shuffled_test_data.shape[0]]

print(np.sum(train_targets), train_sample_count, np.sum(train_targets) / train_sample_count)
print(np.sum(validation_targets), validation_sample_count, np.sum(validation_targets) / validation_sample_count)
#print(np.sum(test_targets), test_sample_count, np.sum(test_targets) / test_sample_count)




215 547 0.3930530164533821
50 136 0.36764705882352944


In [6]:
np.savez('Titanic_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Titanic_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Titanic_data_test', inputs=test_inputs)

npz=[]

In [7]:
class Titanic_Data_Reader():
    

    def __init__(self, dataset, batch_size = None):
    
        npz = np.load('Titanic_data_{0}.npz'.format(dataset))
        
                 
        if npz == 'Titanic_data_test':
                
                self.inputs= npz['inputs'].astype(np.float)
                
        
        else:  
                        self.inputs, self.targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)
        
        
        
        if batch_size is None:
            self.batch_size = self.inputs.shape[0]
        else:
            self.batch_size = batch_size
        self.curr_batch = 0
        self.batch_count = self.inputs.shape[0] // self.batch_size
    
    
    def __next__(self):
        if self.curr_batch >= self.batch_count:
            self.curr_batch = 0
            raise StopIteration()
        
        if npz == 'Titanic_data_test':
            batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
            inputs_batch = self.inputs[batch_slice]
            self.curr_batch += 1
            return inputs_batch
                 
        else:  
                        
            batch_slice = slice(self.curr_batch * self.batch_size, (self.curr_batch + 1) * self.batch_size)
            inputs_batch = self.inputs[batch_slice]
            self.curr_batch += 1
            targets_batch = self.targets[batch_slice]
            self.curr_batch += 1
            classes_num = 2
            targets_one_hot = np.zeros((targets_batch.shape[0], classes_num))
            targets_one_hot[range(targets_batch.shape[0]), targets_batch] = 1
        
        
            return inputs_batch, targets_one_hot
    
        

        
    def __iter__(self):
            return self
    
    

    
    
 

In [8]:
input_size = 6
output_size = 2
hidden_layer_size = 100

tf.reset_default_graph()


inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.int32, [None, output_size])


weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])
outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])
outputs_2 = tf.nn.sigmoid(tf.matmul(outputs_1, weights_2) + biases_2)

weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", [output_size])


outputs = tf.matmul(outputs_2, weights_3) + biases_3


loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=outputs, labels=targets)
mean_loss = tf.reduce_mean(loss)



out_equals_target = tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1))
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))


optimize = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(mean_loss)
sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)



batch_size = 60
max_epochs = 150
prev_validation_loss = 9999999.

 
train_data = Titanic_Data_Reader('train', batch_size)
validation_data = Titanic_Data_Reader('validation')

 
for epoch_counter in range(max_epochs):
    

    curr_epoch_loss = 0.
    
    for input_batch, target_batch in train_data:
        _, batch_loss = sess.run([optimize, mean_loss], 
            feed_dict={inputs: input_batch, targets: target_batch})
        
    
        curr_epoch_loss += batch_loss
    
    curr_epoch_loss /= train_data.batch_count
    
    
    validation_loss = 0.
    validation_accuracy = 0.
    
    for input_batch, target_batch in validation_data:
        validation_loss, validation_accuracy = sess.run([mean_loss, accuracy],
            feed_dict={inputs: input_batch, targets: target_batch})
    
    
    print('Epoch '+str(epoch_counter+1)+
          '. Training loss: '+'{0:.3f}'.format(curr_epoch_loss)+
          '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
          '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
    
    
    if validation_loss > prev_validation_loss:
        break
        
    
    prev_validation_loss = validation_loss
    
print('End of training.')

Epoch 1. Training loss: 0.548. Validation loss: 0.925. Validation accuracy: 63.24%
Epoch 2. Training loss: 0.535. Validation loss: 0.905. Validation accuracy: 63.24%
Epoch 3. Training loss: 0.523. Validation loss: 0.886. Validation accuracy: 63.24%
Epoch 4. Training loss: 0.512. Validation loss: 0.867. Validation accuracy: 63.24%
Epoch 5. Training loss: 0.500. Validation loss: 0.848. Validation accuracy: 63.24%
Epoch 6. Training loss: 0.489. Validation loss: 0.831. Validation accuracy: 63.24%
Epoch 7. Training loss: 0.478. Validation loss: 0.814. Validation accuracy: 63.24%
Epoch 8. Training loss: 0.468. Validation loss: 0.798. Validation accuracy: 63.24%
Epoch 9. Training loss: 0.458. Validation loss: 0.782. Validation accuracy: 63.24%
Epoch 10. Training loss: 0.448. Validation loss: 0.767. Validation accuracy: 63.24%
Epoch 11. Training loss: 0.439. Validation loss: 0.753. Validation accuracy: 63.24%
Epoch 12. Training loss: 0.431. Validation loss: 0.740. Validation accuracy: 63.24%
E

Epoch 104. Training loss: 0.251. Validation loss: 0.519. Validation accuracy: 77.21%
Epoch 105. Training loss: 0.250. Validation loss: 0.519. Validation accuracy: 77.21%
Epoch 106. Training loss: 0.250. Validation loss: 0.519. Validation accuracy: 77.94%
Epoch 107. Training loss: 0.249. Validation loss: 0.519. Validation accuracy: 77.94%
Epoch 108. Training loss: 0.249. Validation loss: 0.519. Validation accuracy: 77.94%
Epoch 109. Training loss: 0.248. Validation loss: 0.519. Validation accuracy: 78.68%
Epoch 110. Training loss: 0.248. Validation loss: 0.518. Validation accuracy: 78.68%
Epoch 111. Training loss: 0.248. Validation loss: 0.518. Validation accuracy: 78.68%
Epoch 112. Training loss: 0.247. Validation loss: 0.518. Validation accuracy: 78.68%
Epoch 113. Training loss: 0.247. Validation loss: 0.518. Validation accuracy: 78.68%
Epoch 114. Training loss: 0.246. Validation loss: 0.518. Validation accuracy: 78.68%
Epoch 115. Training loss: 0.246. Validation loss: 0.518. Validati

In [11]:
test_data=Titanic_Data_Reader('test')
new_targets=[]
for input_batch in test_data:
        test_accuracy = sess.run([ accuracy],
            feed_dict={inputs: input_batch})
        
test_accuracy_percent=test_accuracy[0] * 100
print('Test Accuracy'.format(str(test_accuracy_percent)+'%'))

KeyError: 'targets is not a file in the archive'